In [22]:
# ! pip install fasttext
# ! pip install sacremoses evaluate
! pip install keras

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
from datasets import load_dataset
from transformers import AutoTokenizer
import evaluate
import numpy as np
from transformers import TFAutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import create_optimizer
import keras
import os 
import fasttext
from transformers.keras_callbacks import KerasMetricCallback


In [24]:
import tensorflow as tf
print(tf.__version__)
tf.config.list_physical_devices('GPU')

2.4.1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [81]:
dataset1 = load_dataset('poleval2019_cyberbullying', 'task01')
dataset2 = load_dataset('poleval2019_cyberbullying', 'task02')

Found cached dataset poleval2019_cyberbullying (/home/maciektr/.cache/huggingface/datasets/poleval2019_cyberbullying/task01/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset poleval2019_cyberbullying (/home/maciektr/.cache/huggingface/datasets/poleval2019_cyberbullying/task02/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)


  0%|          | 0/2 [00:00<?, ?it/s]

### Bayesian classifier with TF * IDF weighting.

In [103]:
tf_vectorizer = CountVectorizer() # or term frequency

In [104]:
data_dict = dataset1['train'].to_dict()
data_test_dict = dataset1['test'].to_dict()

In [105]:
X_train_tf = tf_vectorizer.fit_transform(data_dict['text'])
X_test_tf = tf_vectorizer.transform(data_test_dict['text'])

In [106]:
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, data_dict['label'])


MultinomialNB()

In [107]:
y_pred = naive_bayes_classifier.predict(X_test_tf)

In [108]:
score1 = metrics.accuracy_score(data_test_dict['label'], y_pred)
print(metrics.classification_report(data_test_dict['label'], y_pred,
                                            target_names=['No', 'Yes']))


              precision    recall  f1-score   support

          No       0.87      1.00      0.93       866
         Yes       0.73      0.06      0.11       134

    accuracy                           0.87      1000
   macro avg       0.80      0.53      0.52      1000
weighted avg       0.85      0.87      0.82      1000



### Fasttext text classifier

In [73]:
data_path = os.path.abspath('./data.txt')
data_path

'/home/maciektr/Programowanie/AGH_Laby/nlp/6_classification/data.txt'

In [74]:
def fasttext_write_data(data, data_path: str):
    with open(data_path, 'w+') as file:
        data_dict = dataset1['train'].to_dict()
        for text, label in zip(data_dict['text'], data_dict['label']):
            text = text.replace('\n', ' ')
            file.write(f'__label__{label} {text}\n')

In [75]:
fasttext_write_data(dataset1['train'], data_path)

In [80]:
! head -n 5 data.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
__label__0 Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.
__label__0 @anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe
__label__0 @anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.
__label__0 @anonymized_account @anonymized_account Musi. Innej drogi nie mamy.
__label__0 Odrzut natychmiastowy, kwaśna mina, mam problem


In [76]:
classifier_fasttext_data1 = fasttext.train_supervised(input=data_path)

Read 0M words
Number of words:  31486
Number of labels: 2
Progress: 100.0% words/sec/thread:  477823 lr:  0.000000 avg.loss:  0.202303 ETA:   0h 0m 0s


In [82]:
fasttext_write_data(dataset2['train'], data_path)

In [83]:
! head -n 5 data.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
__label__0 Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.
__label__0 @anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe
__label__0 @anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.
__label__0 @anonymized_account @anonymized_account Musi. Innej drogi nie mamy.
__label__0 Odrzut natychmiastowy, kwaśna mina, mam problem


In [78]:
classifier_fasttext_data2 = fasttext.train_supervised(input=data_path)

Read 0M words
Number of words:  31486
Number of labels: 2
Progress: 100.0% words/sec/thread:  478078 lr:  0.000000 avg.loss:  0.206674 ETA:   0h 0m 0s


### Transformer classifier 

In [ ]:
# (take into account that a number of experiments should be performed for this model).

In [29]:
model_name = 'allegro/herbert-base-cased'
# "distilbert-base-uncased"

In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_data1 = dataset1.map(preprocess_function, batched=True)
tokenized_data2 = dataset2.map(preprocess_function, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [31]:
accuracy = evaluate.load("accuracy")


In [32]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

Dataset 1

In [33]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_data1["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_data1["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_data1["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)


In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[metric_callback])

Dataset 2

In [33]:
batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_data2["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_data2["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_data2["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)


In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[metric_callback])